
# Customer Sentiment

In [5]:
#!pip install OpenAI
#!pip install langchain
#!pip install langchain_community
#!pip install Cohere
!pip install langchain-openai langchain-cohere python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1
  Attempting uninstall: langchain-core
    Fo

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [2]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key


In [3]:

import os, json
import pandas as pd



In [6]:

from langchain_openai import ChatOpenAI


In [7]:

CSV_PATH = "https://github.com/giridhar276/genai/raw/refs/heads/main/datasets/Bank_Customer_conversations.csv"
MODEL = "gpt-4o-mini"
TEMPERATURE = 0
TIMEOUT = 60

OUTPUT_PATH = CSV_PATH.replace(".csv", "_with_sentiment_oneprompt.csv")



In [33]:

CSV_PATH = "https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations_2000.csv"
MODEL = "gpt-4o-mini"
TEMPERATURE = 0
TIMEOUT = 200

OUTPUT_PATH = CSV_PATH.replace(".csv", "_with_sentiment_oneprompt_bigdata.csv")

In [34]:

PROMPT = (
    "You are a sentiment classifier and summarize the customer's message. Return ONE LINE of JSON with keys "
    '\"label\", \"confidence\", \"reason\". \"summary\" '
    'Label must be one of "positive", "neutral", "negative". '
    "Confidence is a number in [0,1]. Reason is <= 15 words. Summary is <=100"
    "No extra text before/after JSON.\n\n"
    "Classify the customer text:\n"
    '\"\"\"{text}\"\"\"'
)


In [35]:

df = pd.read_csv(CSV_PATH)
df = df[["customer_text"]].copy()
df["customer_text"] = df["customer_text"].astype(str).str.strip()

print("Rows:", len(df))
df.head(3)


Rows: 2000


,customer_text
0,Ms. Patel: My card ending 0530 got blocked aft...
1,Ms. Patel: I filed a loan application 27-Aug-2...
2,Mr. Singh: I filed a loan application 30-Aug-2...


In [36]:

llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE, timeout=TIMEOUT)

def classify_one(text: str):
    resp = llm.invoke(PROMPT.format(text=text))
    return json.loads(resp.content)  # assumes valid one-line JSON


In [37]:

labels, confs, reasons, summaries = [], [], [], []

for txt in df["customer_text"].tolist():
    obj = classify_one(txt)
    labels.append(obj["label"])
    confs.append(obj["confidence"])
    reasons.append(obj["reason"])
    summaries.append(obj["summary"])


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-6eyZRo8peK1Weuz5ZaUXohoy on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:

df["sentiment_label"] = labels
df["sentiment_confidence"] = confs
df["sentiment_reason"] = reasons
df["summary"] = summaries


In [ ]:

df.to_csv("finalsentiment_v2.csv", index=False)
print(f"Saved: {OUTPUT_PATH}")



In [25]:
df.head()

,customer_text,sentiment_label,sentiment_confidence,sentiment_reason,summary
0,Ms. Patel: My card ending 0530 got blocked aft...,negative,0.85,Frustration with service and request for direc...,Ms. Patel's card is blocked due to wrong PIN a...
1,Ms. Patel: I filed a loan application 27-Aug-2...,negative,0.85,Frustration with service and request for direc...,Ms. Patel is dissatisfied with the loan applic...
2,Mr. Singh: I filed a loan application 30-Aug-2...,negative,0.85,Customer is frustrated with delays and seeks u...,Mr. Singh is inquiring about his loan applicat...
3,Mr. Verma: I filed a loan application 10-Sep-2...,neutral,0.80,Requesting information about loan application ...,Mr. Verma inquires about the status and timeli...
4,Ms. Patel: I submitted an insurance claim 22-A...,negative,0.85,Delay causing inconvenience and issues for plans.,Ms. Patel is frustrated with the delay in her ...


In [26]:
print(df["sentiment_label"].value_counts())


sentiment_label
negative    15
neutral     15
Name: count, dtype: int64


In [27]:
df.shape

(30, 5)